In [3]:
# 절대 임포트 설정
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#필요한 라이브러리 임포트
import argparse
import sys 
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [4]:
FLAGS=None

In [40]:
def deepnn(x):
    """
    숫자를 분류하기위한 dnn그래프를 만든다.
    x:(n_examples,784)
    y(n_examples,10)
    keep_prob:drop outd을 위한 sclaer placeholder이다.

    """ 
    x_image=tf.reshape(x,[-1,28,28,1])  #마지막 차원은 특징들을 나타낸다. 이코드에서 이미지가 gray_sclae이라서 
                                        # 1차원이지만 원래는 채널값때문에 3차원이 된다.
    
    #첫번째 cnn
    w_conv1=weight_variable([5,5,1,32])  # 5X5짜리 합성곱을 하면서 32개의 필터로 맵핑
    b_conv1=bias_variable([32])
    h_conv1=tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)
    #첫번쨰 풀링
    h_pool1=max_pool_2x2(h_conv1)
    
    #두번쨰 cnn 32개의 특징들을 64개의 특징들로 맵핑한다.
    w_conv2=weight_variable([5,5,32,64])
    b_conv2=bias_variable([64])
    h_conv2=tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)
    
    h_pool2=max_pool_2x2(h_conv2)
    
    # fully_connected layer1- 2번의 다운 샘플림이후 우리의 이미지는 7*7*64 특징들이 됨
    # 이를 1024개의 특징들로 맵핑한다.
    w_fc1=weight_variable([7*7*64,1024])
    b_fc1=bias_variable([1024])
    
    h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
    h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)
    
    # drop out - 모델의 복잡도를 컨트롤 하낟. 특징들의 co-adaptation을 방지한다.
    keep_prob=tf.placeholder(tf.float32)
    h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)
    
    # 1024개의 특징들을 10개의 클래스 숫자 0,9로 맵핑한다.
    w_fc2=weight_variable([1024,10])
    b_fc2=bias_variable([10])
    
    y_conv=tf.matmul(h_fc1_drop,w_fc2)+b_fc2
    
    return y_conv,keep_prob

In [41]:
def conv2d(x,w):
    """conv2d는 full stride를 가진 2d convolution layerd를 변환한다. """
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding="SAME")

def max_pool_2x2(x):
    """max_pool_2x2는 특징들을 2x만큼 다운 샘플링한다."""
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

def weight_variable(shape):
    """weight_variable은 주어진 shape에 대한 weight variable을 생성한다."""
    initial=tf.truncated_normal(shape,stddev=0.1)  # truncated_normal은 정규분포에서 난수 생성, stddev는 표준편차를 의미함
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_vaiable는 주어진 shape에 대한 bias_variable을 생성한다."""
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [46]:
def main(_):
    mnist=input_data.read_data_sets(FLAGS.data_dir,one_hot=True)
    
    #모델을 생성한다. 
    x=tf.placeholder(tf.float32,[None,784])
    y_=tf.placeholder(tf.float32,[None,10])
    
    y_conv,keep_prob=deepnn(x)
    
    #에러와 최적화 함수를 만든다.
    cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv))
    train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
    #정확도를 예측한다.
    correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for i in range(2000):
            batch=mnist.train.next_batch(50)
            if i % 100 ==0:
                train_accuracy=accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
                
                print('step %d,train_accuracy %g' % (i, train_accuracy))
            # 50%의 확률을 drop out을 시켜서 학습을 진행한다.     
            train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})
            
            # 테스트 출력
            print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

            


In [48]:
if __name__=='__main__':
    parser=argparse.ArgumentParser()
    parser.add_argument('--data_dir',type=str,
                       default='/tmp/tensorflow/mnist/input_data',
                       help='directory for storing inpur data')
    FLAGS, unparsed=parser.parse_known_args()
    tf.app.run(main=main,argv=[sys.argv[0]]+unparsed)

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
step 0,train_accuracy 0.06
test accuracy 0.102
test accuracy 0.1075
test accuracy 0.0757
test accuracy 0.0569
test accuracy 0.062
test accuracy 0.081
test accuracy 0.1089
test accuracy 0.1447
test accuracy 0.1872
test accuracy 0.2231
test accuracy 0.2637
test accuracy 0.294
test accuracy 0.3134
test accuracy 0.3197
test accuracy 0.32
test accuracy 0.3125
test accuracy 0.3104
test accuracy 0.3211
test accuracy 0.3385
test accuracy 0.3662
test accuracy 0.3938
test accuracy 0.4171
test accuracy 0.4324
test accuracy 0.4402
test accuracy 0.4455
test accuracy 0.4493
test accuracy 0.4554
test accuracy 0.4583
test accuracy 0.4593
test accuracy 0.4673
test accuracy 0.4782
test accuracy 0.4895
test accuracy 0.5079
test

test accuracy 0.9337
test accuracy 0.9341
test accuracy 0.9343
test accuracy 0.9341
test accuracy 0.9346
test accuracy 0.9351
test accuracy 0.9358
test accuracy 0.9361
test accuracy 0.9364
test accuracy 0.9373
test accuracy 0.9372
test accuracy 0.9366
test accuracy 0.9374
test accuracy 0.937
test accuracy 0.9358
test accuracy 0.9348
test accuracy 0.9347
test accuracy 0.9346
test accuracy 0.9352
test accuracy 0.9347
test accuracy 0.9354
test accuracy 0.9373
test accuracy 0.9384
test accuracy 0.9399
test accuracy 0.9402
test accuracy 0.9401
step 400,train_accuracy 0.92
test accuracy 0.94
test accuracy 0.9402
test accuracy 0.9406
test accuracy 0.9395
test accuracy 0.9397
test accuracy 0.9404
test accuracy 0.9408
test accuracy 0.9408
test accuracy 0.9414
test accuracy 0.942
test accuracy 0.9423
test accuracy 0.9416
test accuracy 0.9411
test accuracy 0.9408
test accuracy 0.9409
test accuracy 0.94
test accuracy 0.94
test accuracy 0.9394
test accuracy 0.9389
test accuracy 0.9382
test accuracy

KeyboardInterrupt: 